In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from experiment import rollout, ReplayBuffer, Trajectory, segments_to_training, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('CartPole-v1')

In [3]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward = rollout(episodes=1000, env=env, render=False)
rb.add(trajectories)

print(f"Average Episode Reward: {avg_reward}")

Average Episode Reward: 23.115


In [4]:
loss_object = torch.nn.BCEWithLogitsLoss().to(device)

In [5]:
batch_size = 1024

In [6]:
def train_step(inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets.unsqueeze(1))
    
    loss.backward()
    optimizer.step()
    
    return loss

In [7]:
loss_sum = 0
loss_count = 0

In [8]:
epochs = 1000000
epoch, model, optimizer, loss = load_model(env, device, train=True)

for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(x, y)
    loss_sum += loss
    loss_count += 1
    
    if i % 100 == 0:        
        trajectories, mean_reward = rollout(100, env=env, model=model, sample_action=True, replay_buffer=rb, 
                              device=device)
        rb.add(trajectories)
        
        print(f"Average Episode Reward: {mean_reward}")        

    if i % 200 == 0:
        print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
        save_model(i, model, optimizer, loss)

Average Episode Reward: 9.41
i: 0, Loss: 0.693107008934021
Average Episode Reward: 29.14
Average Episode Reward: 37.0
i: 200, Loss: 0.68612140417099
Average Episode Reward: 36.49
Average Episode Reward: 34.23
i: 400, Loss: 0.6777036786079407
Average Episode Reward: 45.34
Average Episode Reward: 47.23
i: 600, Loss: 0.6718940138816833
Average Episode Reward: 64.29
Average Episode Reward: 70.2
i: 800, Loss: 0.6672784686088562
Average Episode Reward: 52.95
Average Episode Reward: 71.81
i: 1000, Loss: 0.6633278131484985
Average Episode Reward: 69.15
Average Episode Reward: 75.11
i: 1200, Loss: 0.6597281694412231
Average Episode Reward: 45.02
Average Episode Reward: 76.02
i: 1400, Loss: 0.6560639142990112
Average Episode Reward: 66.71
Average Episode Reward: 25.27
i: 1600, Loss: 0.6528140306472778
Average Episode Reward: 64.51
Average Episode Reward: 84.58
i: 1800, Loss: 0.6500992178916931
Average Episode Reward: 67.34
Average Episode Reward: 104.93
i: 2000, Loss: 0.6474925875663757
Average 

KeyboardInterrupt: 

In [9]:
rb.sample_command()

(500.0, 500.0)

In [11]:
cmd = (500, 500) #rb.sample_command()
env = gym.make('CartPole-v1')
e, model, _, l = load_model(train=False, env=env, device=device)
print(f"Loaded model at epoch: {e} with loss {l}")
_, mean_reward = rollout(episodes=3, env=env, model=model, sample_action=False, 
                      replay_buffer=rb, render=True, device=device)


print(f"Average Episode Reward: {mean_reward}")

Loaded model at epoch: 47200 with loss 0.5926135182380676
Average Episode Reward: 500.0
